In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision import transforms
from torch.autograd import Variable

import numpy as np
import pandas as pd
import difflib
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import Dataset, DataLoader

from IPython import display
import string
import dataloader
from dataloader import MoviesDataset
#load data
movies = pd.read_csv('tmdb_5000_movies.csv')
movies = movies.iloc[:-3]
features = ['title', 'genres', 'keywords', 'original_language', 'vote_average']

# make dataset
dataset = MoviesDataset(movies)

#clean null values
for feature in features:
    movies[feature] = movies[feature].fillna('')
    
comb_features = movies['title']+' '+movies['genres']+' '+movies['keywords']+' '+movies['original_language']
vectorizer = TfidfVectorizer()
feat_vectors = vectorizer.fit_transform(comb_features).toarray()
vote_avg = movies['vote_average'].values.reshape(-1, 1)

feat_vectors = np.concatenate([feat_vectors, vote_avg], axis=1)
similarity = cosine_similarity(feat_vectors)
print(similarity.shape)

(4800, 4800)


In [2]:
import network
from network import Reccomender

model_path = "./reccomender_model.pth"
model = Reccomender(1)  # Ensure the architecture matches the saved model
state_dict = torch.load(model_path, map_location=torch.device('cpu'))  # Load the state dictionary
model.load_state_dict(state_dict)  # Load the weights into the model
model.eval()  # Set the model to evaluation mode

#testing
movie = movies.loc[movies['id'] == 1726]
tensor = dataset[movie.index[0]]
tensor = tensor.unsqueeze(0)
tensor = tensor.unsqueeze(1)
print(tensor.shape)
result = model(tensor)
result = result.detach().cpu()
print(result)
print(f"Reccomendation: {dataset.GetSimilarItems(result[0], top_n=5)}")

/local_scratch/slurm.2590444/ipykernel_128545/837863814.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch.device('cp

torch.Size([1, 1, 32])
tensor([[-0.0144,  0.9999, -0.4374, -0.1353, -0.0076,  0.6729, -0.7948,  0.6463,
         -0.7490, -0.3769, -0.9942, -0.9989, -0.9997,  1.0000, -0.9989, -0.7990,
          0.9967,  0.9901, -0.9438, -0.9921, -0.7463, -0.9993, -0.0215, -0.2596,
         -0.9567,  0.9841, -0.9898,  0.8692, -0.9901,  0.4348,  0.1317,  0.9970]])
Reccomendation: ['iron man 3', 'iron man', 'iron man 2', 'xmen apocalypse', 'steel']
